In [1]:
import pydicom
import json
import re
import os
import sys
import shutil
import glob
import random
import subprocess
import pathlib
import yaml
import nibabel as nib
import gzip
import pandas as pd
import numpy as np
import platform
import multiprocessing

In [2]:
# cfg = "C:/Users/smart/Desktop/GitProjects/convsauce/ConvertSource/cfg.test.yml"
cfg = "/Users/brac4g/Desktop/convsauce/ConvertSource/cfg.test.yml"

In [3]:
def read_config(config_file, verbose = False):
    '''
    Reads configuration file and creates a dictionary of search terms for 
    certain modalities provided that BIDS modalities are used as keys. If
    exclusions are provided (via the key 'exclude') then an exclusion list is 
    created. Otherwise, 'exclusion_list' is returned as an empty list. If 
    additional settings are specified, they should be done so via the key
    'metadata' to enable writing of additional metadata. Otherwise, an 
    empty dictionary is returned.
    
    Arguments:
        config_file (string): file path to yaml configuration file.
        verbose (boolean): Prints additional information to screen.
    
    Returns: 
        data_map (dict): Nested dictionary of search terms for BIDS modalities
        exclusion_list (list): List of exclusion terms
        meta_dict (dict): Nested dictionary of metadata terms to write to JSON file(s)
    '''
    
    with open(config_file) as file:
        data_map = yaml.safe_load(file)
        if verbose:
            print("Initialized parameters from configuration file")
        
    if any("exclude" in data_map for element in data_map):
        if verbose:
            print("exclusion option implemented")
        exclusion_list = data_map["exclude"]
        del data_map["exclude"]
    else:
        if verbose:
            print("exclusion option not implemented")
        exclusion_list = list()
        
    if any("metadata" in data_map for element in data_map):
        if verbose:
            print("implementing additional settings for metadata")
        meta_dict = data_map["metadata"]
        del data_map["metadata"]
    else:
        if verbose:
            print("no metadata settings")
        meta_dict = dict()
        
    return data_map,exclusion_list,meta_dict

In [4]:
search_dict, exclusion_list, param_dict = read_config(cfg,True)

Initialized parameters from configuration file
exclusion option implemented
implementing additional settings for metadata


In [5]:
search_dict

{'anat': {'T1w': ['T1', 'T1w', 'TFE'], 'T2w': ['T2', 'T2w', 'TSE']},
 'func': {'bold': {'rest': ['rsfMR', 'rest', 'FFE', 'FEEPI'],
   'visualstrobe': ['vis', 'visual']}},
 'fmap': {'fmap': ['map']},
 'swi': {'swi': ['swi']},
 'dwi': {'dwi': ['diffusion', 'DTI', 'DWI', '6_DIR']}}

In [6]:
exclusion_list

['SURVEY',
 'Reg',
 'SHORT',
 'LONG',
 'MRS',
 'PRESS',
 'DEFAULT',
 'ScreenCapture',
 'PD',
 'ALL',
 'SPECTRO']

In [7]:
param_dict

{'common': {'Manufacturer': 'Philips',
  'ManufacturersModelName': 'Ingenia',
  'MagneticFieldStrength': 3,
  'InstitutionName': "Cincinnati Children's Hospital Medical Center"},
 'func': {'rest': {'ParallelAcquisitionTechnique': 'SENSE',
   'PhaseEncodingDirection': 'j',
   'MultibandAccelerationFactor': 6,
   'TaskName': 'Resting State',
   'dir': 'PA',
   'NumberOfVolumesDiscardedByScanner': 4},
  'visualstrobe': {'PhaseEncodingDirection': 'j',
   'TaskName': 'Visual (Strobe) Task',
   'NumberOfVolumesDiscardedByScanner': 4}},
 'dwi': {'PhaseEncodingDirection': 'j', 'dir': 'PA'},
 'fmap': {'Units': 'Hz'}}

In [8]:
# data_dir_par = "C:/Users/smart/Desktop/GitProjects/convsauce/287H_C10/PAR REC"
data_dir_par = "/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC"

In [9]:
# data_dir_dcm = "C:/Users/smart/Desktop/GitProjects/convsauce/IRC287H-8/20171003"
data_dir_dcm = "/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003"

In [10]:
# data_dir_nii = "C:/Users/smart/Desktop/GitProjects/convsauce/287H_C10/NIFTI"
data_dir_nii = "/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI"

In [11]:
def get_dcm_files(dcm_dir):
    '''
    Creates a file list consisting of the first DICOM file in a parent DICOM directory. 
    A file list is then returned.
    
    Arguments:
        dcm_dir (string): Absolute path to parent DICOM data directory

    Returns: 
        dcm_files (list): List of DICOM filenames, complete with their absolute paths.
    '''
    
    # Create directory list
    dcm_dir = os.path.abspath(dcm_dir)
    parent_dcm_dir = os.path.join(dcm_dir,'*')
    dcm_dir_list = glob.glob(parent_dcm_dir, recursive=True)

    # Initilized dcm_file list
    dcm_files = list()
    
    # Iterate through files in the dicom directory list
    for dir_ in dcm_dir_list:
        # print(dir_)
        for root, dirs, files in os.walk(dir_):
            # print(files[0])
            tmp_dcm_file = files[0] # only need the first dicom file
            tmp_dcm_dir = root
            tmp_file = os.path.join(tmp_dcm_dir, tmp_dcm_file)

            dcm_files.append(tmp_file)
            break

    return dcm_files

In [13]:
def get_dcm_files(dcm_dir):
    '''
    Creates a file list consisting of the first DICOM file in a parent DICOM directory. 
    A file list is then returned.

    Arguments:
        dcm_dir (string): Absolute path to parent DICOM data directory

    Returns: 
        dcm_files (list): List of DICOM filenames, complete with their absolute paths.
    '''

    # Create directory list
    dcm_dir = os.path.abspath(dcm_dir)
    parent_dcm_dir = os.path.join(dcm_dir,'*')
    dcm_dir_list = glob.glob(parent_dcm_dir, recursive=True)

    # Initilized dcm_file list
    dcm_files = list()

    # Iterate through files in the dicom directory list
    for dir_ in dcm_dir_list:
        # print(dir_)
        for root, dirs, files in os.walk(dir_):
            tmp_dcm_file = files[0] # only need the first dicom file
            tmp_dcm_dir = root
            tmp_file = os.path.join(tmp_dcm_dir, tmp_dcm_file)

            dcm_files.append(tmp_file)
            break

    return dcm_files

In [14]:
def create_file_list(data_dir, file_ext="", order="size"):
    '''
    Creates a file list by globbing a directory for a specific file
    extension and sorting by some determined order. A file list is 
    then returned
    
    Arguments:
        data_dir (string): Absolute path to data directory (must be a directory dump of image data)
        file_ext (string): File extension to glob. Built-in options include:
            - 'par' or 'PAR': Searches for PAR headers
            - 'dcm' or 'DICOM': Searches for DICOM directories, then searches for one file from each DICOM directory
            - 'nii', or 'Nifti': Searches for nifti files (including gzipped nifti files)
        order (string): Order to sort the list. Valid options are: 'size' and 'time':
            - 'size': sorts by file size in ascending order (default)
            - 'time': sorts by file modification time in ascending order
            - 'none': no sorting is applied and the list is generated as the system finds the files
    
    Returns: 
        file_list (list): List of filenames, complete with their absolute paths.
    '''
    
    # Check file extension
    if file_ext != "":
        if file_ext.upper() == "PAR" or file_ext.upper() == "REC":
            file_ext = "PAR"
            file_ext = f".{file_ext.upper()}"
        elif file_ext.lower() == "dcm" or file_ext.upper() == "DICOM":
            file_ext = "dcm"
            file_ext = f".{file_ext.lower()}"
        elif file_ext.lower() == "nii" or file_ext.lower() == "nifti":
            file_ext = "nii"
            file_ext = f".{file_ext.lower()}*" # Add wildcard for globbling gzipped files
        else:
            file_ext = f".{file_ext}"
    
    # Check sort order
    if order.lower() == "size":
        order_key = os.path.getsize
    elif order.lower() == "time":
        order_key = os.path.getmtime
    elif order.lower() == "none":
        order_key=None
    else:
        order_key = os.path.getsize
        print("Unrecognized keyword option. Using default.")
    
    # Create file list
    if file_ext == ".dcm":
        file_list = sorted(get_dcm_files(data_dir), key=order_key, reverse=False)
    elif file_ext != ".dcm":
        file_names = os.path.join(data_dir, f"*{file_ext}")
        file_list = sorted(glob.glob(file_names, recursive=True), key=order_key, reverse=False)
    
    return file_list

In [15]:
par_file_list = create_file_list(data_dir=data_dir_par,file_ext="par")
dcm_file_list = create_file_list(data_dir=data_dir_dcm,file_ext="dcm")
nii_file_list = create_file_list(data_dir=data_dir_nii,file_ext="nii")

In [16]:
par_file_list

['/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_2_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_1_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_14_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_16_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_13_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_NEONATAL_SURVEY_17_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_isoReg_-_WIP_DTI_6DIR_B800_12_6.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_faReg_-_WIP_DTI_6DIR_B800_12_4.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_SAG_4_5.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_SAG_15_5.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_AXIAL__4_4.PAR',
 '/Users/brac4g/Desktop/convsauce/2

In [17]:
dcm_file_list

['/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/1101_rsfMRI_MB6_SENSE_1_fat_shift_P_017100310465322437/MR1101027463.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/0_DEFAULT_PS_SERIES_2017100310463791022/PR0000000001.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/1701_WM_SV_PRESS_35_017100311174543840/MR1701000001.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/201_SAG_T1W_3D_Y_INNER_TI_1100_017100310184810020/MR0201000137.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/303_CORONAL_2017100310262626000/MR0303000091.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/0_DEFAULT_PS_SERIES_2017100310374358016/PR0000000001.dcm']

In [18]:
nii_file_list

['/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_Philips_GRE_Map_SENSE_10_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_15_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_4_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_SAG_T1W_3D_Y_INNER_TI_1100_3_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__15_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__15_4.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_SAG_15_5.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__3_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_SAG_4_5.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__4_4.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__4_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI

In [19]:
def file_exclude(file_list, data_dir, exclusion_list = [], verbose = False):
    '''
    Excludes files from the conversion process by removing filenames
    that contain words that match those found in the 'exclusion_list'
    from the 'read_config' function - should any files need/want to be 
    excluded.
    
    If 'exclusion_list' is empty, then the original 'file_list' is returned.
    
    Arguments:
        file_list (list): List of filenames
        data_dir (string): Absolute path to parent directory that contains the image data
        exclusion_list (list): List of words to be matched. Filenames that contain these words will be excluded.
        verbose (bool): Boolean - True or False.
    
    Returns: 
        currated_list (list): Currated list of filenames, with unwanted filenames removed.
    '''
            
    # Check file extension in file list
    if 'dcm' in file_list[0]:
        file_ext = "dcm"
        file_ext = f".{file_ext.lower()}"
    elif 'PAR' in file_list[0]:
        file_ext = "PAR"
        file_ext = f".{file_ext.upper()}"
    elif 'nii' in file_list[0]:
        file_ext = "nii"
        file_ext = f".{file_ext.lower()}*" # Add wildcard for globbling gzipped files
    else:
        file_ext = ""
        file_ext = f".{file_ext.lower()}"
    
    # create set of lists
    file_set = set(file_list)
    
    # create empty sets
    currated_set = set()
    exclusion_set = set()
    
    if len(exclusion_list) == 0:
        currated_set = file_set
    else:
        for file in exclusion_list:
            if file_ext == '.dcm':
                dir_ = os.path.join(data_dir, f"*{file}*",f"*{file_ext}")
            else:
                dir_ = os.path.join(data_dir, f"*{file}*{file_ext}")
            f_names = glob.glob(dir_, recursive=True)        
            f_names_set = set(f_names)
            exclusion_set.update(f_names_set)
            
        currated_set = file_set.difference(exclusion_set)

    currated_list = list(currated_set)
    
    return currated_list

In [20]:
par_file_list_currated = file_exclude(par_file_list,data_dir_par,exclusion_list)
dcm_file_list_currated = file_exclude(dcm_file_list,data_dir_dcm,exclusion_list)
nii_file_list_currated = file_exclude(nii_file_list,data_dir_nii,exclusion_list)

In [21]:
par_file_list_currated

['/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_AXIAL__4_4.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_15_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_AXIAL__15_4.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_rsfMRI_NR1_MB3_SENSE_1_fat_shift_P_8_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_6_DIR_B0_A_TE88_SENSE_NO_MB_NO_4DYN_5_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_DTI_6DIR_B800_12_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_SAG_15_5.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_Philips_GRE_Map_SENSE_10_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_4_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_WIP_SAG_T1W_3D_Y_INNER_TI_1100_3_1.PAR',
 '/Users/brac4g/Desktop/convsauce/287H_C10/PAR REC/287H_C10_W

In [22]:
dcm_file_list_currated

['/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/1101_rsfMRI_MB6_SENSE_1_fat_shift_P_017100310465322437/MR1101027463.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/303_CORONAL_2017100310262626000/MR0303000091.dcm',
 '/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/201_SAG_T1W_3D_Y_INNER_TI_1100_017100310184810020/MR0201000137.dcm']

In [23]:
nii_file_list_currated

['/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__4_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__15_4.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_SAG_T1W_3D_Y_INNER_TI_1100_3_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_4_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_T2W_TSE_AXIAL_NEONATE_NSA1_15_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_WIP_Philips_GRE_Map_SENSE_10_1.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_CORONAL__15_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_SAG_4_5.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__3_3.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_AXIAL__4_4.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI/287H_C10_SAG_15_5.nii.gz',
 '/Users/brac4g/Desktop/convsauce/287H_C10/NIFTI

In [24]:
def str_in_substr(sub_str_,str_):
    '''
    DEPRECATED: Should only be used if config_file uses comma separated
        lists to denote search terms.
    
    Searches a (longer) string using a comma separated string 
    consisting of substrings. Returns 'True' or 'False' if any part
    of the substring is found within the larger string.
    
    Example:
        str_in_substr('T1,TFE','sub_T1_image_file') would return True.
        str_in_substr('T2,TSE','sub_T1_image_file') would return False.
    
    Arguments:
        sub_str_ (string): Substring used for matching.
        str_ (string): Larger string to be searched for matches within substring.
    
    Returns: 
        bool_var (bool): Boolean - True or False
    '''
    
    bool_var = False
    
    for word in sub_str_.split(","):
        if any(word in str_ for element in str_):
            bool_var = True
            
    return bool_var

In [25]:
def list_in_substr(list_,str_):
    '''
    Searches a string using a list that contains substrings. 
    Returns 'True' or 'False' if any elements of the list are 
    found within the string.
    
    Example:
        list_in_substr('['T1','TFE']','sub_T1_image_file') would return True.
        list_in_substr('['T2','TSE']','sub_T1_image_file') would return False.
    
    Arguments:
        list_ (string): list containing strings used for matching.
        str_ (string): Larger string to be searched for matches within substring.
    
    Returns: 
        bool_var (bool): Boolean - True or False
    '''
    
    bool_var = False
    
    for word in list_:
        if any(word.lower() in str_.lower() for element in str_.lower()):
            bool_var = True
            
    return bool_var

In [26]:
def is_valid_dcm(dcm_file, verbose=False):
    '''
    Checks for a valid DICOM file by inspecting the conversion type label in the DICOM file header.
    This field should be blank. If this label is populated, then it is likely a secondary capture image 
    and thus is not likely to contain meaningful image information.
    
    Arguments:
        dcm_file (string): DICOM filename with absolute filepath
        verbose (boolean): Enable verbosity
    
    Returns: 
        is_valid (boolean): True if DICOM file is not a secondary capture (or does not have text in the conversion type label field)
    '''
    
    # Read DICOM file header
    ds = pydicom.dcmread(dcm_file)
    
    # Invalid files include secondary image captures, and are not suitable for 
    # nifti conversion as they are often not converted and cause problems.
    # This string should be empty. If it is populated, then its likely a secondary capture.
    conv_type = ds.ConversionType
    
    if conv_type in '':
        is_valid = True
    else:
        is_valid = False
        if verbose:
            print(f"Please check Conversion Type (0008, 0064) in dicom header. The presented DICOM file is not a valid file: {dcm_file}.")
    
    return is_valid

In [27]:
def get_scan_tech(search_dict, file, json_file=""):
    '''
    Searches DICOM or PAR file header for scan technique/MR modality used in accordance with the search terms provided
    by the nested dictionary.
    
    Note: This function is still undergoing active development.
    
    Arguments:
        search_dict (dict): Nested dictionary from the 'read_config' function
        dcm_file (string): Source image filename with absolute filepath
    
    Returns: 
        None
    '''
    
    # Check file extension in file
    # Perform Scanning Techniqe Search
    if '.dcm' in file.lower():
        get_dcm_scan_tech(search_dict,file)
    elif '.PAR' in file.upper():
        get_par_scan_tech(search_dict,file)
    else:
        print("unknown modality")
        
    return None

In [28]:
def get_dcm_scan_tech(dcm_file, search_dict, keep_unknown=True, verbose=False):
    '''
    Searches DICOM file header for scan technique/MR modality used in accordance with the search terms provided by the
    nested dictionary. The DICOM header field searched is a Philips DICOM private tag (2001,1020) [Scanning Technique 
    Description MR]. In the case that field does not match, is empty, or does not exist, then more common DICOM tags
    are searched - and they include: Series Description, Protocol Name, and Image Type.
    
    Note: This function is still undergoing active development.
    
    Arguments:
        search_dict (dict): Nested dictionary from the 'read_config' function
        dcm_file (string): DICOM filename with absolute filepath
    
    Returns: 
        None
    '''
    
    mod_found = False
    
    # Load DICOM data and read header
    ds = pydicom.dcmread(dcm_file)
    
    # Search DICOM header for Scan Technique used
    dcm_scan_tech_str = str(ds[0x2001,0x1020])
    
    for key,item in search_dict.items():
        for dict_key,dict_item in search_dict[key].items():
            if isinstance(dict_item,list):
                if list_in_substr(dict_item,dcm_scan_tech_str):
                    mod_found = True
                    if verbose:
                        print(f"{key} - {dict_key}: {dict_item}")
                    scan_type = key
                    scan = dict_key
                    if scan_type.lower() == 'dwi':
                        data_to_bids_dwi(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_dwi="",ses="",scan_type=scan_type)
                    elif scan_type.lower() == 'fmap':
                        data_to_bids_fmap(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_fmap="",ses="",scan_type=scan_type)
                    else:
                        data_to_bids_anat(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_anat="",ses="",scan_type=scan_type)
                    if mod_found:
                        break
            elif isinstance(dict_item,dict):
                tmp_dict = search_dict[key]
                for d_key,d_item in tmp_dict[dict_key].items():
                    if list_in_substr(d_item,dcm_scan_tech_str):
                        mod_found = True
                        if verbose:
                            print(f"{key} - {dict_key} - {d_key}: {d_item}")
                        scan_type = key
                        scan = dict_key
                        task = d_key
                        if scan_type.lower() == 'func':
                            data_to_bids_func(bids_out_dir,file,sub,scan,task="",meta_dict_com,meta_dict_func="",ses="",scan_type=scan_type)
                        elif scan_type.lower() == 'dwi':
                            data_to_bids_dwi(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_dwi="",ses="",scan_type=scan_type)
                        elif scan_type.lower() == 'fmap':
                            data_to_bids_fmap(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_fmap="",ses="",scan_type=scan_type)
                        else:
                            data_to_bids_anat(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_anat="",ses="",scan_type=scan_type)
                        if mod_found:
                            break
                            
        if mod_found:
            break
    
    # Secondary look in the case Private Field (2001, 1020) [Scanning Technique Description MR] is empty
    if not mod_found:
        # Define list of DICOM header fields
        dcm_fields = ['SeriesDescription', 'ImageType', 'ProtocolName']
        
        for dcm_field in dcm_fields:
            dcm_scan_tech_str = str(eval(f"ds.{dcm_field}")) # This makes me dangerously uncomfortable
            
            for key,item in search_dict.items():
                for dict_key,dict_item in search_dict[key].items():
                    if isinstance(dict_item,list):
                        if list_in_substr(dict_item,dcm_scan_tech_str):
                            mod_found = True
                            if verbose:
                                print(f"{key} - {dict_key}: {dict_item}")
                            scan_type = key
                            scan = dict_key
                            if scan_type.lower() == 'dwi':
                                data_to_bids_dwi(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_dwi="",ses="",scan_type=scan_type)
                            elif scan_type.lower() == 'fmap':
                                data_to_bids_fmap(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_fmap="",ses="",scan_type=scan_type)
                            else:
                                data_to_bids_anat(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_anat="",ses="",scan_type=scan_type)
                            if mod_found:
                                break
                    elif isinstance(dict_item,dict):
                        tmp_dict = search_dict[key]
                        for d_key,d_item in tmp_dict[dict_key].items():
                            if list_in_substr(d_item,dcm_scan_tech_str):
                                mod_found = True
                                if verbose:
                                    print(f"{key} - {dict_key} - {d_key}: {d_item}")
                                scan_type = key
                                scan = dict_key
                                task = d_key
                                if scan_type.lower() == 'func':
                                    data_to_bids_func(bids_out_dir,file,sub,scan,task="",meta_dict_com,meta_dict_func="",ses="",scan_type=scan_type)
                                elif scan_type.lower() == 'dwi':
                                    data_to_bids_dwi(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_dwi="",ses="",scan_type=scan_type)
                                elif scan_type.lower() == 'fmap':
                                    data_to_bids_fmap(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_fmap="",ses="",scan_type=scan_type)
                                else:
                                    data_to_bids_anat(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_anat="",ses="",scan_type=scan_type)
                                if mod_found:
                                    break

            if mod_found:
                break
                
    if not mod_found:
        if verbose:
            print("unknown modality")
        if keep_unknown:
            scan_type = 'unknown_modality'
            scan = 'unknown'
            data_to_bids_anat(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_anat="",ses="",scan_type=scan_type)
        
    return None

In [29]:
def get_par_scan_tech(par_file, search_dict, keep_unknown=True, verbose=False):
    '''
    Searches PAR file header for scan technique/MR modality used in accordance with the search terms provided by the
    nested dictionary. A regular expression (regEx) search string is defined and searched for conventional PAR headers.
    
    Note: This function is still undergoing active development.
    
    Arguments:
        search_dict (dict): Nested dictionary from the 'read_config' function
        par_file (string): PAR filename with absolute filepath
    
    Returns: 
        None
    '''
    
    mod_found = False
    
    # Define regEx search string
    regexp = re.compile(r'.    Technique                          :  .*', re.M | re.I)
    
    # Open and search PAR header file
    with open(par_file) as f:
        for line in f:
            match_ = regexp.match(line)
            if match_:
                par_scan_tech_str = match_.group()
    
    # Search Scan Technique with search terms
    for key,item in search_dict.items():
        for dict_key,dict_item in search_dict[key].items():
            if isinstance(dict_item,list):
                if list_in_substr(dict_item,par_scan_tech_str):
                    mod_found = True
                    if verbose:
                        print(f"{key} - {dict_key}: {dict_item}")
                    scan_type = key
                    scan = dict_key
                    if scan_type.lower() == 'dwi':
                        data_to_bids_dwi(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_dwi="",ses="",scan_type=scan_type)
                    elif scan_type.lower() == 'fmap':
                        data_to_bids_fmap(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_fmap="",ses="",scan_type=scan_type)
                    else:
                        data_to_bids_anat(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_anat="",ses="",scan_type=scan_type)
                    if mod_found:
                        break
            elif isinstance(dict_item,dict):
                tmp_dict = search_dict[key]
                for d_key,d_item in tmp_dict[dict_key].items():
                    if list_in_substr(d_item,par_scan_tech_str):
                        mod_found = True
                        if verbose:
                            print(f"{key} - {dict_key} - {d_key}: {d_item}")
                        scan_type = key
                        scan = dict_key
                        task = d_key
                        if scan_type.lower() == 'func':
                            data_to_bids_func(bids_out_dir,file,sub,scan,task="",meta_dict_com,meta_dict_func="",ses="",scan_type=scan_type)
                        elif scan_type.lower() == 'dwi':
                            data_to_bids_dwi(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_dwi="",ses="",scan_type=scan_type)
                        elif scan_type.lower() == 'fmap':
                            data_to_bids_fmap(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_fmap="",ses="",scan_type=scan_type)
                        else:
                            data_to_bids_anat(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_anat="",ses="",scan_type=scan_type)
                        if mod_found:
                            break
                            
        if mod_found:
            break
            
    if not mod_found:
        if verbose:
            print("unknown modality")
        if keep_unknown:
            scan_type = 'unknown_modality'
            scan = 'unknown'
            data_to_bids_anat(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_anat="",ses="",scan_type=scan_type)
        
    return None

In [30]:
def convert_modality(file, search_dict, verbose=False):
    '''
    Converts an image file and extracts information from the filename (such as the modality). 
    
    Note: This function is still undergoing active development.
    Note: Add support for extra dictionaries
    
    Arguments:
        search_dict (dict): Nested dictionary from the 'read_config' function
        file (string): Filename with absolute filepath
        verbose (boolean): Enable verbosity
    
    Returns: 
        None
    '''
    
    mod_found = False
    
    # Check file type
    if 'nii' in file:
        file_ext = "nii"
        file_ext = f".{file_ext.lower()}"
    elif 'dcm' in file:
        file_ext = "dcm"
        file_ext = f".{file_ext.lower()}"
        if not is_valid_dcm(file,verbose):
            sys.exit(f"Invalid DICOM file. Please check {file}")
    
    for key,item in search_dict.items():
        for dict_key,dict_item in search_dict[key].items():
            if isinstance(dict_item,list):
                if list_in_substr(dict_item,file):
                    mod_found = True
                    if verbose:
                        print(f"{key} - {dict_key}: {dict_item}")
                    scan_type = key
                    scan = dict_key
                    if scan_type.lower() == 'dwi':
                        data_to_bids_dwi(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_dwi="",ses="",scan_type=scan_type)
                    elif scan_type.lower() == 'fmap':
                        data_to_bids_fmap(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_fmap="",ses="",scan_type=scan_type)
                    else:
                        data_to_bids_anat(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_anat="",ses="",scan_type=scan_type)
                    if mod_found:
                        break
            elif isinstance(dict_item,dict):
                tmp_dict = search_dict[key]
                for d_key,d_item in tmp_dict[dict_key].items():
                    if list_in_substr(d_item,file):
                        mod_found = True
                        if verbose:
                            print(f"{key} - {dict_key} - {d_key}: {d_item}")
                        scan_type = key
                        scan = dict_key
                        task = d_key
                        if scan_type.lower() == 'func':
                            data_to_bids_func(bids_out_dir,file,sub,scan,task="",meta_dict_com,meta_dict_func="",ses="",scan_type=scan_type)
                        elif scan_type.lower() == 'dwi':
                            data_to_bids_dwi(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_dwi="",ses="",scan_type=scan_type)
                        elif scan_type.lower() == 'fmap':
                            data_to_bids_fmap(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_fmap="",ses="",scan_type=scan_type)
                        else:
                            data_to_bids_anat(bids_out_dir,file,sub,scan,meta_dict_com,meta_dict_anat="",ses="",scan_type=scan_type)
                        if mod_found:
                            break
                        
    if not mod_found:
        get_scan_tech(search_dict,file)
    
    return None

In [31]:
def batch_convert(file_list, dictionary, verbose=False):
    '''
    Batch conversion function for image files. 
    
    Note: This function is still undergoing active development.
    
    Arguments:
        file_list (list): List of filenames with absolute filepaths
        dictionary (dict): Nested dictionary from the 'read_config' function
        verbose (boolean): Enable verbosity
    
    Returns: 
        None
    '''
    
    for file in file_list:
        try:
            convert_modality(dictionary,file,verbose)
        except SystemExit:
            pass
    
    return None

In [30]:
batch_convert(par_file_list_currated,search_dict,verbose=True)

swi - swi: ['swi']
func - bold - rest: ['rsfMR', 'rest', 'FFE', 'FEEPI']
anat - T2w: ['T2', 'T2w', 'TSE']
anat - T2w: ['T2', 'T2w', 'TSE']
anat - T2w: ['T2', 'T2w', 'TSE']
anat - T2w: ['T2', 'T2w', 'TSE']
dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']
anat - T2w: ['T2', 'T2w', 'TSE']
func - bold - rest: ['rsfMR', 'rest', 'FFE', 'FEEPI']
anat - T2w: ['T2', 'T2w', 'TSE']
dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']
dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']
dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']
anat - T2w: ['T2', 'T2w', 'TSE']
anat - T1w: ['T1', 'T1w', 'TFE']
fmap - fmap: ['map']
dwi - dwi: ['diffusion', 'DTI', 'DWI', '6_DIR']
anat - T2w: ['T2', 'T2w', 'TSE']
anat - T1w: ['T1', 'T1w', 'TFE']
func - bold - rest: ['rsfMR', 'rest', 'FFE', 'FEEPI']
anat - T1w: ['T1', 'T1w', 'TFE']


In [125]:
batch_convert(dcm_file_list_currated,search_dict,verbose=True)

anat - T1w: ['T1', 'T1w', 'TFE']
func - bold - rest: ['rsfMR', 'rest', 'FFE', 'FEEPI']
anat - T2w: ['T2', 'T2w', 'TSE']


#### `TaskName` JSON file appending funtion

In [4]:
# task_name = ""
task_name = "visualstrobe"

In [5]:
if task_name == "":
    print("task_name is empty")
else:
    print(f"task_name is: {task_name}")

task_name is: visualstrobe


In [1]:
# nii_file = "c:/Users/smart/Desktop/GitProjects/convsauce/BIDS/rawdata/sub-C10/ses-001/func/sub-C10_ses-001_task-rest_acq-PA_run-01_bold.nii.gz"
nii_file = "/Users/brac4g/Desktop/convsauce/BIDS/rawdata/sub-C10/ses-001/func/sub-C10_ses-001_task-rest_acq-PA_run-01_bold.nii.gz"

In [2]:
import os
import nibabel as nib
import numpy as np

In [3]:
os.path.exists(nii_file)

True

In [4]:
img = nib.load(nii_file)

In [5]:
img

In [6]:
img.header.get_data_shape()

(64, 64, 45, 400)

In [7]:
type(img.header.get_data_shape())

tuple

In [8]:
dims = img.header.get_data_shape()

In [9]:
dims

(64, 64, 45, 400)

In [10]:
frames = dims[3]

In [11]:
frames

400

In [12]:
def get_num_frames(nii_file):
    '''
    working doc-string
    '''
    
    img = nib.load(nii_file)
    dims = img.header.get_data_shape()
    num_frames = dims[3]
    
    return num_frames

In [13]:
get_num_frames(nii_file)

400

# `NifTi` File Conversion Functions

In [32]:
def convert_anat(file,work_dir,work_name):
    '''
    Converts raw anatomical (and functional) MR images to NifTi file format, with a BIDS JSON sidecar.
    Returns a NifTi file and a JSON sidecar (file) by globbing an isolated directory.
    
    Arguments:
        file (string): Absolute filepath to raw image data
        work_dir (string): Working directory
        work_name (string): Output file name
        
    Returns:
        nii_file (string): Absolute file path to NifTi image
        json_file (string): Absolute file path to JSON sidecar
    '''
    
    # Convert (anatomical) iamge data
    convert_image_data(file, work_name, work_dir)
    
    # Get files
    dir_path = os.path.join(work_dir, basename)
    nii_file = glob.glob(dir_path + '*.nii*')
    json_file = glob.glob(dir_path + '*.json')
    
    # Convert lists to strings
    nii_file = ''.join(nii_file)
    json_file = ''.join(json_file)
    
    return nii_file, json_file

In [33]:
def convert_dwi(file,work_dir,work_name):
    '''
    Converts raw diffusion weigthed MR images to NifTi file format, with a BIDS JSON sidecar.
    Returns a NifTi file, JSON sidecar (file), and (FSL-style) bval and bvec files by globbing 
    an isolated directory.
    
    Arguments:
        file (string): Absolute filepath to raw image data
        work_dir (string): Working directory
        work_name (string): Output file name
        
    Returns:
        nii_file (string): Absolute file path to NifTi image
        json_file (string): Absolute file path to JSON sidecar
        bval (string): Absolute file path to bval file
        bvec (string): Absolute file path to bvec file
    '''
    
    # Convert diffusion iamge data
    convert_image_data(file, work_name, work_dir)
    
    # Get files
    dir_path = os.path.join(out_dir, basename)
    nii_file = glob.glob(dir_path + '*.nii*')
    json_file = glob.glob(dir_path + '*.json')
    bval = glob.glob(dir_path + '*.bval*')
    bvec = glob.glob(dir_path + '*.bvec*')
    
    # Convert lists to strings
    nii_file = ''.join(nii_file)
    json_file = ''.join(json_file)
    bval = ''.join(bval)
    bvec = ''.join(bvec)
    
    return nii_file, json_file, bval, bvec

In [34]:
def convert_fmap(file,work_dir,work_name):
    '''
    Converts raw precomputed fieldmap MR images to NifTi file format, with a BIDS JSON sidecar.
    Returns two NifTi files, and their corresponding JSON sidecars (files), by globbing an isolated directory.
    
    N.B.: This function is mainly designed to handle fieldmap data case 3 from bids-specifications document. Furhter support for 
    the additional cases requires test/validation data. 
    BIDS-specifications document located here: 
    https://github.com/bids-standard/bids-specification/blob/master/src/04-modality-specific-files/01-magnetic-resonance-imaging-data.md
    
    Arguments:
        file (string): Absolute filepath to raw image data
        work_dir (string): Working directory
        work_name (string): Output file name
        
    Returns:
        nii_fmap (string): Absolute file path to NifTi image fieldmap
        json_fmap (string): Absolute file path to corresponding JSON sidecar
        nii_mag (string): Absolute file path to NifTi magnitude image
        json_mag (string): Absolute file path to corresponding JSON sidecar
    '''
    
    # Convert diffusion iamge data
    convert_image_data(file, work_name, work_dir)
    
    # Get files
    dir_path = os.path.join(out_dir, basename)
    nii_fmap = glob.glob(dir_path + '*real*.nii*')
    json_fmap = glob.glob(dir_path + '*real*.json')
    nii_mag = glob.glob(dir_path + '.nii*')
    json_mag = glob.glob(dir_path + '.json')
    
    # Convert lists to strings
    nii_fmap = ''.join(nii_real)
    json_fmap = ''.join(json_real)
    nii_mag = ''.join(nii_mag)
    json_mag = ''.join(json_mag)

    return nii_fmap, json_fmap, nii_mag, json_mag

In [35]:
def data_to_bids_anat(bids_out_dir, file, sub, scan, meta_dict_com=dict(), meta_dict_anat=dict(), ses=1, scan_type='anat'):
    '''
    Renames converted NifTi-2 files to conform with the BIDS naming convension (in the case of anatomical files).
    This function accepts any image file (DICOM, PAR REC, and NifTi-2). If the image file is a raw data file (e.g. DICOM, PAR REC)
    it is converted to NifTi first, then renamed. The output BIDS directory need not exist at runtime.
    
    Arguments:
        bids_out_dir (string): Path to output BIDS directory. 
        file (string): Filepath to image file.
        sub (int or string): Subject ID
        scan (string): Modality (e.g. T1w, T2w, or SWI)
        meta_dict_com (dict): Metadata dictionary for common image metadata
        meta_dict_anat (dict): Metadata dictionary for common anatomical image specific metadata
        ses (int or string): Session ID
        scan_type (string): BIDS sub-directory scan type. Valid options include, but are not limited to: anat (default), func, fmap, dwi, etc.
        
    Returns:
        out_nii (string): Absolute filepath to gzipped output NifTi-2 file
        out_json (string): Absolute filepath to corresponding JSON file
    '''

    # Create Output Directory Variables
    # Zeropad subject ID if possible
    try:
        ses = '{:03}'.format(int(ses))
    except ValueError:
        pass
    # Zeropad session ID if possible
    try:
        ses = '{:03}'.format(int(ses))
    except ValueError:
        pass
    
    out_dir = os.path.join(bids_out_dir, f"sub-{sub}", f"ses-{ses}", f"{scan_type}")

    # Make output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Get absolute filepaths
    bids_out_dir = os.path.abspath(bids_out_dir)
    out_dir = os.path.abspath(out_dir)
    
    # Create temporary output names/directories
    n = 10000 # maximum N for random number generator
    tmp_out_dir = os.path.join(out_dir, f"sub-{sub}", 'tmp_dir' + str(random.randint(0, n)))
    tmp_basename = 'tmp_basename' + str(random.randint(0, n))
    
    if not os.path.exists(tmp_out_dir):
        os.makedirs(tmp_out_dir)

    # Convert image file
    # Check file extension in file
    if '.nii.gz' in file:
        nii_file = cp_file(file, tmp_out_dir, tmp_basename)
        [path,filename] = file_parts(file)
        json_file = os.path.join(path,filename + '.json')
        try:
            json_file = cp_file(json_file, tmp_out_dir, tmp_basename)
        except FileNotFoundError:
            json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
            pass
    elif '.nii' in file:
        nii_file = cp_file(file, tmp_out_dir, tmp_basename)
        nii_file = gzip_file(nii_file)
        json_file = os.path.join(path,filename + '.json')
        try:
            json_file = cp_file(json_file, tmp_out_dir, tmp_basename)
        except FileNotFoundError:
            json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
            pass
    elif '.dcm' in file or '.PAR' in file:
        [nii_file, json_file] = convert_anat(file,tmp_out_dir,tmp_basename)
    else:
        [nii_file, json_file] = convert_anat(file,tmp_out_dir,tmp_basename)
    
    # Get additional sequence/modality parameters
    if os.path.exists(json_file):
        meta_dict_params = get_data_params(file, json_file)
    else:
        tmp_json = ""
        meta_dict_params = get_data_params(file, tmp_json)
    
    # Update JSON file
    info = dict()
    info = dict_multi_update(info,**meta_dict_com)
    info = dict_multi_update(info,**meta_dict_params)
    info = dict_multi_update(info,**meta_dict_anat)
    
    json_file = update_json(json_file,info)
    
    nii_file = os.path.abspath(nii_file)
    json_file = os.path.abspath(json_file)

    # Append w to T1/T2 if not already done
    if scan in 'T1' or scan in 'T2':
        scan = scan + 'w'

    # Query dictionary for acquisition/naming keys
    try:
        acq = info['acq']
    except KeyError:
        pass
    try:
        ce = info['ce']
    except KeyError:
        pass
    try:
        rec = info['rec']
    except KeyError:
        pass
    
    # Create output filename
    out_name = f"sub-{sub}" + f"_ses-{sub}"
    name_run_dict = dict()

    if acq:
        out_name = out_name + f"_acq-{acq}"
        tmp_dict = {"acq":f"{acq}"}
        name_run_dict.update(tmp_dict)

    if ce:
        out_name = out_name + f"_ce-{ce}"
        tmp_dict = {"ce":f"{ce}"}
        name_run_dict.update(tmp_dict)

    if rec:
        out_name = out_name + f"_rec-{rec}"
        tmp_dict = {"rec":f"{rec}"}
        name_run_dict.update(tmp_dict)
        
    # Get Run number
    run = get_num_runs(outdir, scan=scan, **name_run_dict)
    run = '{:02}'.format(run)

    if run:
        out_name = out_name + f"_run-{run}"

    out_name = out_name + f"_{scan}"


    out_nii = os.path.join(out_dir, out_name + '.nii.gz')
    out_json = os.path.join(out_dir, out_name + '.json')

    os.rename(nii_file, out_nii)
    os.rename(json_file, out_json)

    # remove temporary directory and leftover files
    shutil.rmtree(tmp_out_dir)
    
    return out_nii,out_json

In [36]:
def data_to_bids_func(bids_out_dir, file, sub, scan, task = 'rest', meta_dict_com=dict(), meta_dict_func=dict(), ses=1, scan_type='func'):
    '''
    Renames converted NifTi-2 files to conform with the BIDS naming convension (in the case of functional files).
    This function accepts any image file (DICOM, PAR REC, and NifTi-2). If the image file is a raw data file (e.g. DICOM, PAR REC)
    it is converted to NifTi first, then renamed. The output BIDS directory need not exist at runtime.
    
    Arguments:
        bids_out_dir (string): Path to output BIDS directory. 
        file (string): Filepath to image file.
        sub (int or string): Subject ID
        scan (string): Modality (e.g. bold or cbv)
        task (string): Task for the fMR image data
        meta_dict_com (dict): Metadata dictionary for common image metadata
        meta_dict_func (dict): Metadata dictionary for common functional image specific metadata
        ses (int or string): Session ID
        scan_type (string): BIDS sub-directory scan type. Valid options include, but are not limited to: anat, func (default), fmap, dwi, etc.
        
    Returns:
        out_nii (string): Absolute filepath to gzipped output 4D NifTi-2 file
        out_json (string): Absolute filepath to corresponding JSON file
    '''

    # Create Output Directory Variables
    # Zeropad subject ID if possible
    try:
        ses = '{:03}'.format(int(ses))
    except ValueError:
        pass
    # Zeropad session ID if possible
    try:
        ses = '{:03}'.format(int(ses))
    except ValueError:
        pass
    
    out_dir = os.path.join(bids_out_dir, f"sub-{sub}", f"ses-{ses}", f"{scan_type}")

    # Make output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Get absolute filepaths
    bids_out_dir = os.path.abspath(bids_out_dir)
    out_dir = os.path.abspath(out_dir)
    
    # Create temporary output names/directories
    n = 10000 # maximum N for random number generator
    tmp_out_dir = os.path.join(out_dir, f"sub-{sub}", 'tmp_dir' + str(random.randint(0, n)))
    tmp_basename = 'tmp_basename' + str(random.randint(0, n))
    
    if not os.path.exists(tmp_out_dir):
        os.makedirs(tmp_out_dir)

    # Convert image file
    # Check file extension in file
    if '.nii.gz' in file:
        nii_file = cp_file(file, tmp_out_dir, tmp_basename)
        [path,filename] = file_parts(file)
        json_file = os.path.join(path,filename + '.json')
        try:
            json_file = cp_file(json_file, tmp_out_dir, tmp_basename)
        except FileNotFoundError:
            json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
            pass
    elif '.nii' in file:
        nii_file = cp_file(file, tmp_out_dir, tmp_basename)
        nii_file = gzip_file(nii_file)
        json_file = os.path.join(path,filename + '.json')
        try:
            json_file = cp_file(json_file, tmp_out_dir, tmp_basename)
        except FileNotFoundError:
            json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
            pass
    elif '.dcm' in file or '.PAR' in file:
        [nii_file, json_file] = convert_anat(file,tmp_out_dir,tmp_basename)
    else:
        [nii_file, json_file] = convert_anat(file,tmp_out_dir,tmp_basename)
    
    # Get additional sequence/modality parameters
    if os.path.exists(json_file):
        meta_dict_params = get_data_params(file, json_file)
    else:
        tmp_json = ""
        meta_dict_params = get_data_params(file, tmp_json)
    
    # Update JSON file
    info = dict()
    info = dict_multi_update(info,**meta_dict_com)
    info = dict_multi_update(info,**meta_dict_params)
    info = dict_multi_update(info,**meta_dict_func)
    
    json_file = update_json(json_file,info)
    
    nii_file = os.path.abspath(nii_file)
    json_file = os.path.abspath(json_file)
    
    # Decide if file is 4D timeseries or single-band reference
    num_frames = get_num_frames(nii_file)
    if num_frames == 1:
        scan = 'sbref'

    # Query dictionary for acquisition/naming keys
    try:
        acq = info['acq']
    except KeyError:
        pass
    try:
        ce = info['ce']
    except KeyError:
        pass
    try:
        direction = info['dir']
    except KeyError:
        pass
    try:
        rec = info['rec']
    except KeyError:
        pass
    try:
        echo = info['echo']
    except KeyError:
        pass
    
    # Create output filename    
    out_name = f"sub-{sub}" + f"_ses-{ses}" + f"_task-{task}"
    
    name_run_dict = dict()
    tmp_dict = {"task":f"{task}"}
    name_run_dict.update(tmp_dict)

    if acq:
        out_name = out_name + f"_acq-{acq}"
        tmp_dict = {"acq":f"{acq}"}
        name_run_dict.update(tmp_dict)

    if ce:
        out_name = out_name + f"_ce-{ce}"
        tmp_dict = {"ce":f"{ce}"}
        name_run_dict.update(tmp_dict)

    if direction:
        out_name = out_name + f"_dir-{direction}"
        tmp_dict = {"dirs":f"{direction}"}
        name_run_dict.update(tmp_dict)

    if rec:
        out_name = out_name + f"_rec-{rec}"
        tmp_dict = {"rec":f"{rec}"}
        name_run_dict.update(tmp_dict)
        
    if echo:
        tmp_dict = {"echo":f"{echo}"}
        name_run_dict.update(tmp_dict)
        
    # Get Run number
    run = get_num_runs(outdir, scan=scan, **name_run_dict)
    run = '{:02}'.format(run)

    if run:
        out_name = out_name + f"_run-{run}"

    if echo:
        out_name = out_name + f"_echo-{echo}"

    out_name = out_name + f"_{scan}"


    out_nii = os.path.join(out_dir, out_name + '.nii.gz')
    out_json = os.path.join(out_dir, out_name + '.json')

    os.rename(nii_file, out_nii)
    os.rename(json_file, out_json)

    # remove temporary directory and leftover files
    shutil.rmtree(tmp_out_dir)
    
    return out_nii,out_json

In [37]:
def data_to_bids_fmap(bids_out_dir, file, sub, scan='magnitude', meta_dict_com=dict(), meta_dict_fmap=dict(), ses=1, scan_type='fmap'):
    '''
    Renames converted NifTi-2 files to conform with the BIDS naming convension (in the case of fieldmap files).
    This function accepts any image file (DICOM, PAR REC, and NifTi-2). If the image file is a raw data file (e.g. DICOM, PAR REC)
    it is converted to NifTi first, then renamed. The output BIDS directory need not exist at runtime.
    
    N.B.: This function is mainly designed to handle fieldmap data case 3 from bids-specifications document. Furhter support for 
    the additional cases requires test/validation data. 
    BIDS-specifications document located here: 
    https://github.com/bids-standard/bids-specification/blob/master/src/04-modality-specific-files/01-magnetic-resonance-imaging-data.md
    
    Arguments:
        bids_out_dir (string): Path to output BIDS directory. 
        file (string): Filepath to image file.
        sub (int or string): Subject ID
        scan (string): Modality (e.g. fieldmap, magnitude, or phasediff)
        meta_dict_com (dict): Metadata dictionary for common image metadata
        meta_dict_fmap (dict): Metadata dictionary for common fieldmap image specific metadata
        ses (int or string): Session ID
        scan_type (string): BIDS sub-directory scan type. Valid options include, but are not limited to: anat, func, fmap (default), dwi, etc.
        
    Returns:
        out_nii_fmap (string): Absolute filepath to gzipped output NifTi-2 fieldmap image file
        out_nii_mag (string): Absolute filepath to gzipped output NifTi-2 magnitude image file
        out_json_fmap (string): Absolute filepath to correspond fieldmap image JSON sidecare
        out_json_mag (string): Absolute filepath to correspond magnitude image JSON sidecare
    '''

    # Create Output Directory Variables
    # Zeropad subject ID if possible
    try:
        ses = '{:03}'.format(int(ses))
    except ValueError:
        pass
    # Zeropad session ID if possible
    try:
        ses = '{:03}'.format(int(ses))
    except ValueError:
        pass
    
    out_dir = os.path.join(bids_out_dir, f"sub-{sub}", f"ses-{ses}", f"{scan_type}")

    # Make output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Get absolute filepaths
    bids_out_dir = os.path.abspath(bids_out_dir)
    out_dir = os.path.abspath(out_dir)
    
    # Create temporary output names/directories
    n = 10000 # maximum N for random number generator
    tmp_out_dir = os.path.join(out_dir, f"sub-{sub}", 'tmp_dir' + str(random.randint(0, n)))
    tmp_basename = 'tmp_basename' + str(random.randint(0, n))
    
    if not os.path.exists(tmp_out_dir):
        os.makedirs(tmp_out_dir)

    # Convert image file
    # Check file extension in file
    if '.nii.gz' in file:
        nii_file = cp_file(file, tmp_out_dir, tmp_basename)
        [path,filename] = file_parts(file)
        json_file = os.path.join(path,filename + '.json')
        try:
            json_file = cp_file(json_file, tmp_out_dir, tmp_basename)
        except FileNotFoundError:
            json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
            pass
    elif '.nii' in file:
        nii_file = cp_file(file, tmp_out_dir, tmp_basename)
        nii_file = gzip_file(nii_file)
        json_file = os.path.join(path,filename + '.json')
        try:
            json_file = cp_file(json_file, tmp_out_dir, tmp_basename)
        except FileNotFoundError:
            json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
            pass
    elif '.dcm' in file or '.PAR' in file:
        [nii_fmap, json_fmap, nii_mag, json_mag] = convert_fmap(file,tmp_out_dir,tmp_basename)
    else:
        [nii_fmap, json_fmap, nii_mag, json_mag] = convert_fmap(file,tmp_out_dir,tmp_basename)
    
    # Get additional sequence/modality parameters
    if os.path.exists(json_fmap):
        meta_dict_params = get_data_params(file, json_fmap)
    else:
        tmp_json = ""
        meta_dict_params = get_data_params(file, tmp_json)
    
    # Update JSON file
    info = dict()
    info = dict_multi_update(info,**meta_dict_com)
    info = dict_multi_update(info,**meta_dict_params)
    info = dict_multi_update(info,**meta_dict_fmap)
    
    json_fmap = update_json(json_fmap,info)
    json_mag = update_json(json_mag,info)
    
    nii_fmap = os.path.abspath(nii_fmap)
    nii_mag = os.path.abspath(nii_mag)
    
    json_fmap = os.path.abspath(json_fmap)
    json_mag = os.path.abspath(json_mag)

    # Query dictionary for acquisition/naming keys
    try:
        acq = info['acq']
    except KeyError:
        pass
    
    # Create output filename    
    out_name = f"sub-{sub}" + f"_ses-{ses}"
    name_run_dict = dict()

    if acq:
        out_name = out_name + f"_acq-{acq}"
        tmp_dict = {"acq":f"{acq}"}
        name_run_dict.update(tmp_dict)
        
    # Get Run number
    run = get_num_runs(outdir, scan=scan, **name_run_dict)
    run = '{:02}'.format(run)

    if run:
        out_name = out_name + f"_run-{run}"

    out_name = out_name + f"_{scan}"
    
    out_nii_fmap = os.path.join(out_dir, out_name + '_fieldmap' + '.nii.gz')
    out_nii_mag = os.path.join(out_dir, out_name + '_magnitude' + '.nii.gz')
    
    out_json_fmap = os.path.join(out_dir, out_name + '_fieldmap' + '.json')
    out_json_mag = os.path.join(out_dir, out_name + '_magnitude' + '.json')

    os.rename(nii_fmap, out_nii_fmap)
    os.rename(nii_mag, out_nii_mag)
    
    os.rename(json_fmap, out_json_fmap)
    os.rename(json_mag, out_json_mag)

    # Remove temporary directory and leftover files
    shutil.rmtree(tmp_out_dir)
    
    return out_nii_fmap, out_nii_mag, out_json_fmap, out_json_mag

In [38]:
def data_to_bids_dwi(bids_out_dir, file, sub, scan='dwi', meta_dict_com=dict(), meta_dict_dwi=dict(), ses=1, scan_type='dwi'):
    '''
    Renames converted NifTi-2 files to conform with the BIDS naming convension (in the case of diffuion image files).
    This function accepts any image file (DICOM, PAR REC, and NifTi-2). If the image file is a raw data file (e.g. DICOM, PAR REC)
    it is converted to NifTi first, then renamed. The output BIDS directory need not exist at runtime. If the original
    data format is NifTi, bval and bvec files will be copied over should they exist, otherwise, they will not be
    generated.
    
    Arguments:
        bids_out_dir (string): Path to output BIDS directory. 
        file (string): Filepath to image file.
        sub (int or string): Subject ID
        scan (string): Modality (e.g. dwi, dki, etc)
        meta_dict_com (dict): Metadata dictionary for common image metadata
        meta_dict_dwi (dict): Metadata dictionary for common diffusion image specific metadata
        ses (int or string): Session ID
        scan_type (string): BIDS sub-directory scan type. Valid options include, but are not limited to: anat, func, fmap, dwi (default), etc.
        
    Returns:
        out_nii (string): Absolute filepath to gzipped output diffusion weighted NifTi-2 file
        out_json (string): Absolute filepath to corresponding JSON file
        out_bval (string): Absolute filepath to corresponding b-values file
        out_bvec (string): Absolute filepath to corresponding b-vectors file
    '''

    # Create Output Directory Variables
    # Zeropad subject ID if possible
    try:
        ses = '{:03}'.format(int(ses))
    except ValueError:
        pass
    # Zeropad session ID if possible
    try:
        ses = '{:03}'.format(int(ses))
    except ValueError:
        pass
    
    out_dir = os.path.join(bids_out_dir, f"sub-{sub}", f"ses-{ses}", f"{scan_type}")

    # Make output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Get absolute filepaths
    bids_out_dir = os.path.abspath(bids_out_dir)
    out_dir = os.path.abspath(out_dir)
    
    # Create temporary output names/directories
    n = 10000 # maximum N for random number generator
    tmp_out_dir = os.path.join(out_dir, f"sub-{sub}", 'tmp_dir' + str(random.randint(0, n)))
    tmp_basename = 'tmp_basename' + str(random.randint(0, n))
    
    if not os.path.exists(tmp_out_dir):
        os.makedirs(tmp_out_dir)

    # Convert image file
    # Check file extension in file
    if '.nii.gz' in file:
        nii_file = cp_file(file, tmp_out_dir, tmp_basename)
        [path,filename] = file_parts(file)
        json_file = os.path.join(path,filename + '.json')
        bval = os.path.join(path,filename + '.bval*')
        bvec = os.path.join(path,filename + '.bvec*')
        try:
            json_file = cp_file(json_file, tmp_out_dir, tmp_basename)
            bval = cp_file(bval, tmp_out_dir, tmp_basename)
            bvec = cp_file(bvec, tmp_out_dir, tmp_basename)
        except FileNotFoundError:
            json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
            bval = os.path.join(tmp_out_dir, tmp_basename + '.bval')
            bvec = os.path.join(tmp_out_dir, tmp_basename + '.bvec')
            pass
    elif '.nii' in file:
        nii_file = cp_file(file, tmp_out_dir, tmp_basename)
        nii_file = gzip_file(nii_file)
        json_file = os.path.join(path,filename + '.json')
        bval = os.path.join(path,filename + '.bval*')
        bvec = os.path.join(path,filename + '.bvec*')
        try:
            json_file = cp_file(json_file, tmp_out_dir, tmp_basename)
            bval = cp_file(bval, tmp_out_dir, tmp_basename)
            bvec = cp_file(bvec, tmp_out_dir, tmp_basename)
        except FileNotFoundError:
            json_file = os.path.join(tmp_out_dir, tmp_basename + '.json')
            bval = os.path.join(tmp_out_dir, tmp_basename + '.bval')
            bvec = os.path.join(tmp_out_dir, tmp_basename + '.bvec')
            pass
    elif '.dcm' in file or '.PAR' in file:
        [nii_file, json_file, bval, bvec] = convert_dwi(file,tmp_out_dir,tmp_basename)
    else:
        [nii_file, json_file, bval, bvec] = convert_dwi(file,tmp_out_dir,tmp_basename)
    
    # Get additional sequence/modality parameters
    if os.path.exists(json_file):
        meta_dict_params = get_data_params(file, json_file, bval)
    else:
        tmp_json = ""
        meta_dict_params = get_data_params(file, tmp_json, bval)
    
    # Update JSON file
    info = dict()
    info = dict_multi_update(info,**meta_dict_com)
    info = dict_multi_update(info,**meta_dict_params)
    info = dict_multi_update(info,**meta_dict_dwi)
    
    json_file = update_json(json_file,info)
    
    nii_file = os.path.abspath(nii_file)
    json_file = os.path.abspath(json_file)
    
    bval = os.path.abspath(bval)
    bvec = os.path.abspath(bvec)
    
    # Decide if file is 4D timeseries or single-band reference
    num_frames = get_num_frames(nii_file)
    if num_frames == 1:
        scan = 'sbref'

    # Query dictionary for acquisition/naming keys
    try:
        acq = info['acq']
    except KeyError:
        pass
    try:
        direction = info['dir']
    except KeyError:
        pass
    
    # Non-standard acquisition/naming keys
    # Used in order to differentiate between DWI scans for multiple bvalues
    try:
        bvals = info['bval']
    except KeyError:
        pass
    try:
        echo_time = info['EchoTime']
    except KeyError:
        pass
    
    # Create output filename    
    out_name = f"sub-{sub}" + f"_ses-{ses}"
    name_run_dict = dict()
    
    if bvals:
        vals = ""
        for val in bvals:
            vals = vals + 'b' + val
    
    if bvals and acq and echo_time:
        out_name = out_name + f"_acq-{acq}{vals}TE{echo_time}"
        tmp_dict = {"acq":f"{acq}{vals}TE{echo_time}"}
    elif bvals and acq:
        out_name = out_name + f"_acq-{acq}{vals}"
        tmp_dict = {"acq":f"{acq}{vals}"}
    elif bvals and echo_time:
        out_name = out_name + f"_acq-{vals}TE{echo_time}"
        tmp_dict = {"acq":f"{vals}TE{echo_time}"}
    elif acq and echo_time:
        out_name = out_name + f"_acq-{acq}TE{echo_time}"
        tmp_dict = {"acq":f"{acq}TE{echo_time}"}
    elif acq:
        out_name = out_name + f"_acq-{acq}"
        tmp_dict = {"acq":f"{acq}"}
    elif bvals:
        out_name = out_name + f"_acq-{vals}"
        tmp_dict = {"acq":f"{vals}"}
    elif echo_time:
        out_name = out_name + f"_acq-TE{echo_time}"
        tmp_dict = {"acq":f"TE{echo_time}"}
    else:
        tmp_dict = dict()
        
    name_run_dict.update(tmp_dict)

    if direction:
        out_name = out_name + f"_dir-{direction}"
        tmp_dict = {"dirs":f"{direction}"}
        name_run_dict.update(tmp_dict)
        
    # Get Run number
    run = get_num_runs(outdir, scan=scan, **name_run_dict)
    run = '{:02}'.format(run)

    if run:
        out_name = out_name + f"_run-{run}"

    out_name = out_name + f"_{scan}"


    out_nii = os.path.join(out_dir, out_name + '.nii.gz')
    out_json = os.path.join(out_dir, out_name + '.json')
    
    out_bval = os.path.join(out_dir, out_name + '.bval')
    out_bvec = os.path.join(out_dir, out_name + '.bvec')

    os.rename(nii_file, out_nii)
    os.rename(json_file, out_json)
    
    if bval:
        os.rename(bval,out_bval)
        
    if bvec:
        os.rename(bvec,out_bvec)

    # remove temporary directory and leftover files
    shutil.rmtree(tmp_out_dir)
    
    return out_nii,out_json,out_bval,out_bvec

In [39]:
def get_data_params(file,json_file="", bval_file=""):
    '''
    Creates a dictionary of key mapped parameter items that are often not written to the BIDS JSON sidecar
    when converting Philips DICOM and PAR REC files.
    
    Arguments:
        file (string): Absolute filepath to raw image data file (DICOM or PAR REC)
        json_file (string, optional): Corresponding JSON sidecare file
        bval_file (string, optional): Corresponding bval file for DWI acquisitions
    
    Returns:
        info (dict): Dictionary of key mapped items/values
    '''
    
    # Create empty dictionary
    tmp_dict = dict()
    
    # Check and write bvalue(s) to file
    if bval_file:
        bval_list = get_bvals(bval_file)
        tmp_dict.update({"bval":bval_list})
    
    # Check file type
    if '.dcm' in file.lower():
        red_fact = get_red_fact(file)
        mb = get_mb(file)
        scan_time = get_scan_time(file)
        [eff_echo_sp, tot_read_time]  = calc_read_time(file,json_file)
        source_format = "DICOM"
        tmp_dict.update({"ParallelReductionFactorInPlane": red_fact,
                         "MultibandAccelerationFactor": mb,
                         "EffectiveEchoSpacing": eff_echo_sp,
                         "TotalReadoutTime": tot_read_time,
                         "AcquisitionDuration": scan_time,
                         "SourceDataFormat": source_format})
    elif 'PAR' in file.upper():
        wfs = get_wfs(file)
        red_fact = get_red_fact(file)
        mb = get_mb(file)
        scan_time = get_scan_time(file)
        etl = get_etl(file)
        [eff_echo_sp, tot_read_time]  = calc_read_time(file,json_file)
        source_format = "PAR REC"
        tmp_dict.update({"WaterFatShift": wfs,
                         "ParallelAcquisitionTechnique": 'SENSE',
                         "ParallelReductionFactorInPlane": red_fact,
                         "MultibandAccelerationFactor": mb,
                         "EffectiveEchoSpacing": eff_echo_sp,
                         "TotalReadoutTime": tot_read_time,
                         "AcquisitionDuration": scan_time,
                         "EchoTrainLength": etl,
                         "SourceDataFormat": source_format})
    elif 'nii' in file.lower():
        tr = get_nii_tr(file)
        source_format = "NIFTI"
        tmp_dict.update({"RepetitionTime": tr,
                         "SourceDataFormat": source_format})
    else:
        pass
        
    info = dict()
    info.update(tmp_dict)
    
    return info

# `DICOM` Parameter Functions

In [40]:
def get_red_fact(dcm_file):
    '''
    Extracts parallel reduction factor in-plane value (GRAPPA/SENSE) from file description in the DICOM 
    header for MR scanners. This reduction factor is assumed to be 1 if a value cannot be found from witin
    the DICOM header.
    
    Arguments:
        dcm_file (string): Absolute filepath to DICOM file
        
    Returns:
        red_fact (float): parallel reduction factor in-plane value (e.g. SENSE factor)
    '''

    # Load dicom data
    ds = pydicom.dcmread(dcm_file)
    red_fact = ""
    
    # Get Info
    try:
        red_fact = ds[0x0018,0x9069]
    except KeyError:
        pass
    
    # Get image descriptor
    if not red_fact:
        line = ds.SeriesDescription
        match = re.search(r'SENSE .*?([0-9.-]+)', line, re.M | re.I)
        if match:
            red_fact = match.group(1)
            red_fact = float(red_fact)
        else:
            red_fact = float(1)

    return red_fact

In [41]:
def get_mb(dcm_file):
    '''
    Extracts multi-band acceleration factor from file description in the DICOM header for philips MR scanners.
    
    N.B.: This is done via a regEx search as no DICOM tag stores this information explicitly.
    
    Arguments:
        dcm_file (string): Absolute filepath to DICOM file
        
    Returns:
        mb (int): multi-band acceleration factor
    '''

    # Initialize mb to 1
    mb = 1

    # Load dicom data
    ds = pydicom.dcmread(dcm_file)

    # Get image descriptor
    line = ds.SeriesDescription
    match = re.search(r'MB.*?([0-9.-]+)', line, re.M | re.I)
    if match:
        mb = match.group(1)
        mb = int(mb)

    return mb

In [42]:
def get_scan_time(dcm_file):
    '''
    Gets the acquisition duration (scan time, in s) from the DICOM header.
    
    Arguments:
        dcm_file (string): Absolute filepath to DICOM file
        
    Returns:
        scan_time (float): acquisition duration (scan time, in s)
    '''

    # Load data
    ds = pydicom.dcmread(dcm_file)

    # Gets scan time
    try:
        scan_time = ds.AcquisitionDuration
    except AttributeError:
        pass
        scan_time = 'unknown'

    return scan_time

In [44]:
def get_echo(json_file):
    '''
    Reads the echo time (TE) from the NifTi JSON sidecar and returns it.

    Arguments:
        json_file (string): Absolute path to JSON sidecar

    Returns:
        echo (float): Returns the echo time as a float.
    '''

    with open(json_file, "r") as read_file:
        data = json.load(read_file)

    echo = data.get("EchoTime")

    return echo

# `PAR REC` Parameter Functions

In [45]:
def get_etl(par_file):
    '''
    Gets EPI factor (Echo Train Length) from Philips' PAR Header.
    
    N.B.: This is done via a regEx search as the PAR header is not assumed to change significantly between scanners.
    
    Arguments:
        par_file (string): Absolute filepath to PAR header file
        
    Returns:
        etl (float): Echo Train Length
    '''
    regexp = re.compile(r'.    EPI factor        <0,1=no EPI>     :   .*?([0-9.-]+)')  # Search string for RegEx
    with open(par_file) as f:
        for line in f:
            match = regexp.match(line)
            if match:
                etl = match.group(1)
                etl = int(etl)
    return etl

In [46]:
def get_wfs(par_file):
    '''
    Gets Water Fat Shift from Philips' PAR Header.
    
    N.B.: This is done via a regEx search as the PAR header is not assumed to change significantly between scanners.
    
    Arguments:
        par_file (string): Absolute filepath to PAR header file
        
    Returns:
        wfs (float): Water Fat Shift
    '''
    regexp = re.compile(
        r'.    Water Fat shift \[pixels\]           :   .*?([0-9.-]+)')  # Search string for RegEx, escape the []
    with open(par_file) as f:
        for line in f:
            match = regexp.match(line)
            if match:
                wfs = match.group(1)
                wfs = float(wfs)
    return wfs

In [47]:
def get_red_fact(par_file):
    '''
    Extracts parallel reduction factor in-plane value (SENSE) from the file description in the PAR REC header 
    for Philips MR scanners. This reduction factor is assumed to be 1 if a value cannot be found from witin
    the PAR REC header.
    
    N.B.: This is done via a regEx search as the PAR header is not assumed to change significantly between scanners.
    
    Arguments:
        par_file (string): Absolute filepath to PAR header file
        
    Returns:
        red_fact (float): parallel reduction factor in-plane value (e.g. SENSE factor)
    '''
    
    # Read file
    red_fact = ""
    regexp = re.compile(r' SENSE *?([0-9.-]+)')
    with open(par_file) as f:
        for line in f:
            match = regexp.search(line)
            if match:
                red_fact = match.group(1)
                red_fact = float(red_fact)
            else:
                red_fact = float(1)

    return red_fact

In [48]:
def get_mb(par_file):
    '''
    Extracts multi-band acceleration factor from from Philips' PAR Header.
    
    N.B.: This is done via a regEx search as the PAR header does not normally store this value.
    
    Arguments:
        par_file (string): Absolute filepath to PAR header file
        
    Returns:
        mb (int): multi-band acceleration factor
    '''

    # Initialize mb to 1
    mb = 1
    
    regexp = re.compile(r' MB *?([0-9.-]+)')
    with open(par_file) as f:
        for line in f:
            match = regexp.search(line)
            if match:
                mb = match.group(1)
                mb = int(mb)

    return mb

In [49]:
def get_scan_time(par_file):
    '''
    Gets the acquisition duration (scan time, in s) from the PAR header.
    
    N.B.: This is done via a regEx search as the PAR header is not assumed to change significantly between scanners.
    
    Arguments:
        par_file (string): Absolute filepath to PAR header file
        
    Returns:
        scan_time (float or string): Acquisition duration (scan time, in s). If not in header, return is a string 'unknown'
    '''
    
    scan_time = 'unknown'
    
    regexp = re.compile(
        r'.    Scan Duration \[sec\]                :   .*?([0-9.-]+)')  # Search string for RegEx, escape the []
    with open(par_file) as f:
        for line in f:
            match = regexp.match(line)
            if match:
                scan_time = match.group(1)
                scan_time = float(scan_time)
    return scan_time

    return scan_time

In [50]:
def get_num_runs(out_dir,scan,ses="",task="",acq="",ce="",dirs="",rec="",echo=""):
    '''
    Determines run number of a scan (e.g. T1w, T2w, bold, dwi etc.) in an output directory by globbing the 
    directory for the number of NifTis of the same scan.

    Arguments (required):
        out_dir (string): Absolute path to output directory
        scan (string): Modality (e.g. T1w, T2w, bold, dwi, etc.)

    Arguments (optional):
        ses (string): Session ID
        task (string): Task ID
        acq (string): Acquisition ID
        ce (string): Contrast Enhanced ID
        dirs (string): Directions ID string
        rec (string): Reconstruction algorithm string
        echo (int or string): Echo number from multi-echo functional scan

    Returns:
        run_num (int): Returns the run number for the specific scan
    '''

    runs = os.path.join(out_dir, f"*{ses}*{task}*{acq}*{ce}*{dirs}*{rec}*{echo}*{scan}*.nii*")
    run_num = len(glob.glob(runs))
    run_num = run_num + 1

    return run_num

In [51]:
def cp_file(file,work_dir="",work_name=""):
    '''
    Copies a file. Primarily intended for copying single file image data.
    
    Arguments:
        file (string): File path to source (image) file
        work_dir (string): Absolute path to working directory (must exist at runtime prior to invoakation of this function). If left empty, then the directory of the source file is used.
        work_name (string): Output name for (image) file. If left empty, the output name is the same as the source file.
        
    Returns:
        out_file (string): Absolute path to output file.
    '''
    
    [path, filename, ext] = file_parts(file)
    
    if work_dir == "":
        work_dir = path
    else:
        work_dir = os.path.abspath(work_dir)
        
    if work_name == "":
        work_name = filename
        
    out_file = os.path.join(work_dir,work_name + ext)
    
    shutil.copy(file,out_file)
    
    return out_file

In [52]:
def get_nii_tr(nii_file):
    '''
    Reads the NifTi file header and returns the repetition time (TR, sec) as a value if it is not zero, otherwise this 
    function returns the string 'unknown'.
    
    Arguments:
        nii_file (string): NifTi image filename with absolute filepath
        
    Returns: 
        tr (float or string): Repetition time (TR, sec), if not zero, otherwise 'unknown' is returned.
    '''
    
    # Load nifti file
    img = nib.load(nii_file)
    
    # Store nifti image TR
    tr = float(img.header['pixdim'][4])
    
    # Check if TR is likely
    if tr != 0:
        pass
    else:
        tr = "unknown"
    
    return tr

In [53]:
def file_parts(file):
    '''
    Divides file with file path into: path, filename, extension.
    
    Arguments:
        file (string): File with absolute filepath
        
    Returns: 
        path (string): Path of input file
        filename (string): Filename of input file, without the extension
        ext (string): Extension of input file
    '''
    
    [path, file_with_ext] = os.path.split(file)
    [filename,ext] = os.path.splitext(file_with_ext)
    
    path = str(path)
    filename = str(filename)
    ext = str(ext)
    
    return path,filename,ext

In [54]:
def gzip_file(file,rm_orig=True):
    '''
    Gzips file.
    
    Arguments:
        file (string): Input file
        rm_orig (boolean): If true (default), removes original file
        
    Returns: 
        out_file (string): Gzipped file
    '''
    
    # Define tempory file for I/O buffer stream
    tmp_file = file
    path,f_name_,ext_ = file_parts(tmp_file)
    f_name = f_name_ + ext_ + ".gz"
    out_file = os.path.join(path,f_name)
    
    # Gzip file
    with open(file,"rb") as in_file:
        data = in_file.read(); in_file.close()
        with gzip.GzipFile(out_file,"wb") as tmp_out:
            tmp_out.write(data)
            tmp_out.close()
            
    if rm_orig:
        os.remove(file)
            
    return out_file

In [55]:
def gunzip_file(file,rm_orig=True):
    '''
    Gunzips file.
    
    Arguments:
        file (string): Input file
        rm_orig (boolean): If true (default), removes original file
        
    Returns: 
        out_file (string): Gunzipped file
    '''
    
    # Define tempory file for I/O buffer stream
    tmp_file = file
    path,f_name_,ext_ = file_parts(tmp_file)
    f_name = f_name_ # + ext_[:-3]
    out_file = os.path.join(path,f_name)
    
    with gzip.GzipFile(file,"rb") as in_file:
        data = in_file.read(); in_file.close()
        with open(out_file,"wb") as tmp_out:
            tmp_out.write(data)
            tmp_out.close()
            
    if rm_orig:
        os.remove(file)
    
    return out_file

In [56]:
def update_json(json_file,dictionary):
    '''
    Updates JavaScript Object Notation (JSON) file. If the file does not exist, it is created once
    this function is invoked.
    
    Arguments:
        json_file (string): Input file
        dictionary (dict): Dictionary of key mapped items to write to JSON file
        
    Returns: 
        json_file (string): Updated JSON file
    '''
    
    # Check if JSON file exists, if not, then create JSON file
    if not os.path.exists(json_file):
        with open(json_file,"w"): pass
        
    # Read JSON file
    # Try-Except statement has empty exception as JSONDecodeError is not a valid exception to pass, 
    # thus throwing a name error
    try:
        with open(json_file) as file:
            data_orig = json.load(file)
    except:
        pass
        data_orig = dict()
        
    # Update original data from JSON file
    data_orig.update(dictionary)
    
    # Write updated JSON file
    with open(json_file,"w") as file:
        json.dump(data_orig,file,indent=4)
        
    return json_file

In [57]:
def dict_multi_update(dictionary,**kwargs):
    '''
    Updates a dictionary multiple times depending on the number key word mapped pairs that are provided and 
    returns a separate updated dictionary. The dictionary passed as an argument must exist prior to this 
    function being invoked.
    
    Example usage:
    
        new_dict = dict_multi_update(old_dict,
                                    Manufacturer="Philips",
                                    ManufacturersModelName="Ingenia",
                                    MagneticFieldStrength=3,
                                    InstitutionName="CCHMC")
    
    Arguments:
        dictionary (dict): Dictionary of key mapped items to write to JSON file
        **kwargs (string, key,value pairs): key=value pairs
        
    Returns: 
        new_dict (dict): New updated dictionary
    '''
    
    # Create new dictionary
    new_dict = dictionary.copy()
    
    for key,item in kwargs.items():
        tmp_dict = {key:item}
        new_dict.update(tmp_dict)
        
    return new_dict

In [58]:
def get_metadata(dictionary,scan_type="",task=""):
    '''
    Reads the metadata dictionary and looks for keywords to indicate what metadata should be written to which
    dictionary. For example, the keyword 'common' is used to indicate the common information for the imaging
    protocol and may contain information such as: field strength, phase encoding direction, institution name, etc.
    Additional keywords that are BIDS sub-directories names (e.g. anat, func, dwi) will return an additional
    dictionary which contains metadata specific for those modalities. Func also has additional keywords based on
    the task specified.
    
    Arguments:
        dictionary (dict): Nest dictionary of key mapped items from the 'read_config' function
        scan_type (string): BIDS scan type (e.g. anat, func, dwi, etc., default="")
        task (string): Task name to search in the key mapped dictionary
        
    Returns: 
        com_param_dict (dict): Common parameters dictionary
        scan_param_dict (dict): Scan/modality type parameters dictionary
    '''
    
    # Create empty dictionaries
    com_param_dict = dict()
    scan_param_dict = dict()
    scan_task_dict = dict()
    
    # Iterate through, looking for key words (e.g. common and scan_type)
    for key,item in dictionary.items():
        if key.lower() in 'common':
            com_param_dict = dictionary[key]

        if key.lower() in scan_type:
            scan_param_dict = dictionary[key]
            if task.lower() in scan_param_dict:
                for dict_key,dict_item in scan_param_dict.items():
                    if task.lower() in dict_key:
                        scan_task_dict = scan_param_dict[dict_key]
                        
        if len(scan_task_dict) != 0:
            scan_param_dict = scan_task_dict
    
    return com_param_dict, scan_param_dict 

In [59]:
def get_bvals(bval_file):
    '''
    Reads the bvals from the (FSL-style) bvalue file and returns a list of unique non-zero bvalues
    
    Arguments:
        bval_file (string): Absolute filepath to bval (.bval) file
        
    Returns: 
        bvals_list (list): List of unique, non-zero bvalues.
    '''
    
    vals = np.loadtxt(bval_file)
    vals_nonzero = vals[vals.astype(bool)]
    bvals_list = list(np.unique(vals_nonzero))
    
    return bvals_list

## Calculate `EffectiveEchoSpacing` and `TotalReadoutTime`
-----

See this source for details:https://github.com/bids-standard/bids-specification/blob/master/src/04-modality-specific-files/01-magnetic-resonance-imaging-data.md

-----
Siemens:        

`BWPPPE` = `BandwidthPerPixelPhaseEncode `           

`EffectiveEchoSpacing` = 1 / [`BWPPPE` * `ReconMatrixPE`]          

`TotalReadoutTime` = `EffectiveEchoSpacing * (ReconMatrixPE - 1)`

Philips:

`EffectiveEchoSpacing` = (((1000*`WFS`)/(434.215*(`EchoTrainLength`+1)))/`acceleration`)           

`TotalReadoutTime` = 0.001 * `EffectiveEchoSpacing` * `EchoTrainLength`

See these links for Philips specific details:                 

https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=fsl;162ab1a3.1308           

https://support.brainvoyager.com/brainvoyager/functional-analysis-preparation/29-pre-processing/78-epi-distortion-correction-echo-spacing-and-bandwidth           

https://neurostars.org/t/consolidating-epi-echo-spacing-and-readout-time-for-philips-scanner/4406            







In [210]:
# f = "C:/Users/smart/Desktop/GitProjects/convsauce/IRC287H-8/20171003/1101_rsfMRI_MB6_SENSE_1_fat_shift_P_017100310465322437/MR1101000016.dcm"
# f = "/Users/brac4g/Desktop/convsauce/IRC287H-8/20171003/1101_rsfMRI_MB6_SENSE_1_fat_shift_P_017100310465322437/MR1101000016.dcm"
f = "/Users/brac4g/Downloads/MR.1.3.12.2.1107.5.2.19.45307.2017051015422162853047250.dcm"

In [211]:
ds = pydicom.dcmread(f)
# ds = pydicom.dcmread(dcm_file_list_currated[0])
ds

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'MOSAIC']
(0008, 0012) Instance Creation Date              DA: '20170510'
(0008, 0013) Instance Creation Time              TM: '154222.295000'
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.2.19.45307.2017051015422162853047250
(0008, 0020) Study Date                          DA: '20170510'
(0008, 0021) Series Date                         DA: '20170510'
(0008, 0022) Acquisition Date                    DA: '20170510'
(0008, 0023) Content Date                        DA: '20170510'
(0008, 0030) Study Time                          TM: '152243.068000'
(0008, 0031) Series Time                         TM: '153725.445000'
(0008, 0032) Acquisition Time                    TM: '154158.857500'
(0008, 0033) Content Time                        TM: '154222.295

In [212]:
ds[0x0019, 0x1028]

(0019, 1028) [BandwidthPerPixelPhaseEncode]      FD: 29.481

In [221]:
t = str(ds[0x0019, 0x1028])
t

'(0019, 1028) [BandwidthPerPixelPhaseEncode]      FD: 29.481'

In [219]:
ds.dir

<bound method Dataset.dir of (0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'MOSAIC']
(0008, 0012) Instance Creation Date              DA: '20170510'
(0008, 0013) Instance Creation Time              TM: '154222.295000'
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.2.19.45307.2017051015422162853047250
(0008, 0020) Study Date                          DA: '20170510'
(0008, 0021) Series Date                         DA: '20170510'
(0008, 0022) Acquisition Date                    DA: '20170510'
(0008, 0023) Content Date                        DA: '20170510'
(0008, 0030) Study Time                          TM: '152243.068000'
(0008, 0031) Series Time                         TM: '153725.445000'
(0008, 0032) Acquisition Time                    TM: '154158.857500'
(0008, 0033) Content Time          

In [60]:
def get_bwpppe(dcm_file):
    '''
    Reads the Bandwidth Per Pixel PhaseEncode value from a DICOM header. 
    
    Note: This DICOM field is usually left blank on Philips DICOM headers.
    
    Arguments:
        dcm_file (string): Absolute filepath to DICOM file
        
    Returns:
        bwpppe (float or string): Bandwidth Per Pixel PhaseEncode value
    '''
    
    # Load data
    ds = pydicom.dcmread(dcm_file)
    
    # Get relevant DICOM field
    try:
        val_str = str(ds[0x0019, 0x1028])
        val_list = val_str.split(" ")
        bwpppe = val_list[-1]
        bwpppe = float(bwpppe)
    except (AttributeError,KeyError):
        bwpppe = 'unknown'
        pass
    
    return bwpppe

In [61]:
def get_recon_mat(json_file):
    '''
    Reads ReconMatrixPE (reconstruction matrix phase encode) value from the JSON sidecar.
    
    Arguments:
        json_file (string): Absolute filepath to JSON file
        
    Returns:
        recon_mat (float or string): Recon Matrix PE value
    '''
    
    # Read JSON file
    # Try-Except statement has empty exception as JSONDecodeError is not a valid exception to pass, 
    # thus throwing a name error
    try:
        with open(json_file, "r") as read_file:
            data = json.load(read_file)
            recon_mat = data["ReconMatrixPE"]
    except:
        recon_mat = 'unknown'
        pass
    
    return recon_mat

In [62]:
def get_pix_band(json_file):
    '''
    Reads pixel bandwidth value from the JSON sidecar.
    
    Arguments:
        json_file (string): Absolute filepath to JSON file
        
    Returns:
        pix_band (float or string): Pixel bandwidth value
    '''
    
    # Read JSON file
    # Try-Except statement has empty exception as JSONDecodeError is not a valid exception to pass, 
    # thus throwing a name error
    try:
        with open(json_file, "r") as read_file:
            data = json.load(read_file)
            pix_band = data["PixelBandwidth"]
    except:
        pix_band = 'unknown'
        pass
    
    return pix_band

In [65]:
# (Theoretical) Echo Space:
# 1/BW

In [66]:
# Siemens
1/2440

0.0004098360655737705

In [67]:
# Philips
1/1539

0.000649772579597141

In [68]:
# Effective Echo Spacing Siemens
1/(29.481*64)

0.0005300023744106374

In [69]:
# Total Readout time Siemens
(1/(29.481*64))*(64-1)

0.033390149587870156

In [73]:
# Siemens effective echo spacing with approaches 3 and 4
((1/(2440*64))*(64-1))

0.00040343237704918035

In [81]:
# Siemens total readout time with approaches 3 and 4
(0.00040343237704918035)*(64-1)

0.025416239754098364

In [78]:
# percent error effective echo spacing
((abs(0.0005300023744106374 - 0.00040343237704918035))/0.00040343237704918035) * 100

31.37328696502401

In [79]:
# percent error total readout time
((abs(0.033390149587870156 - 0.025416239754098364)/0.025416239754098364)) * 100

31.37328696502401

In [ ]:
# There approx. 30% error. Maybe try using a fudge factor of some sort, perhaps?

In [80]:
# Siemens effective echo spacing with approaches 3 and 4 (with 30% fudge factor)
((1/(2440*64))*(64-1)) * 1.3

0.0005244620901639344

In [82]:
# Siemens total readout time with approaches 3 and 4
(0.0005244620901639344)*(64-1)

0.03304111168032787

In [83]:
# percent error effective echo spacing
((abs(0.0005300023744106374 - 0.0005244620901639344))/0.00040343237704918035) * 100

1.3732869650240196

In [84]:
# percent error total readout time
((abs(0.033390149587870156 - 0.03304111168032787)/0.025416239754098364)) * 100

1.3732869650240276

In [70]:
# Effective Echo Spacing Philips dicom (not reasonable)
1/(1539*71)

9.15172647319917e-06

In [71]:
# Effective Echo spacing (reasonable, approaches 3 and 4)
((1/(1539*71))*(71-1))

0.0006406208531239419

In [72]:
# Total Readout time philips dicom (reasonable, approaches 3 and 4...acceptable?)
(0.0006406208531239419)*(71-1)

0.04484345971867593

In [90]:
def calc_read_time(file, json_file=""):
    '''
    Calculates the effective echo spacing and total readout time provided several combinations of parameters.
    Several approaches and methods to calculating the effective echo spacing and total readout within this function
    differ and are dependent on the parameters found within the provided JSON sidecar. Currently, there a four 
    approaches for calculating the effective echo space (all with differing values) and two ways of calculating 
    the total readout time. It should also be noted that several of these approaches are vendor specific (e.g. at 
    the time of writing, 16 Jan 2019, the necessary information for approach 1 is only found in Siemens DICOM 
    headers - the necessary information for approach 2 is only possible if the data is stored in PAR REC format as the
    WaterFatShift is a private tag in the Philips DICOM header - approaches 3 and 4 are intended for Philips/GE DICOMs 
    as those values are anticipated to exist in their DICOM headers).
    
    The approaches are listed below:
    
    Approach 1 (BIDS method, Siemens):
        BWPPPE = BandwidthPerPixelPhaseEncode
        EffectiveEchoSpacing = 1/[BWPPPE * ReconMatrixPE]
        TotalReadoutTime = EffectiveEchoSpacing * (ReconMatrixPE - 1)
        
    Approach 2 (Philips method - PAR REC):
        EffectiveEchoSpacing = (((1000 * WaterFatShift)/(434.215 * (EchoTrainLength + 1)))/ParallelReductionFactorInPlane)
        TotalReadoutTime = 0.001 * EffectiveEchoSpacing * EchoTrainLength
    
    Approach 3 (Philips/GE method - DICOM):
        EffectiveEchoSpacing = ((1/(PixelBandwidth * EchoTrainLength)) * (EchoTrainLength - 1)) * 1.3
        TotalReadoutTime = EffectiveEchoSpacing * (EchoTrainLength - 1)
        
    Approach 4 (Philips/GE method - DICOM):
        EffectiveEchoSpacing = ((1/(PixelBandwidth * ReconMatrixPE)) * (ReconMatrixPE - 1)) * 1.3
        tot_read_time = EffectiveEchoSpacing * (ReconMatrixPE - 1)
        
        Note: EchoTrainLength is assumed to be equal to ReconMatrixPE for approaches 3 and 4, as these values are generally close.
        Note: Approaches 3 and 4 appear to have about a 30% decrease in Siemens data when this was tested. The solution was to implement a fudge factor that accounted for the 30% decrease.
    
    Arguments:
        file (string): Absolute filepath to raw image data file (DICOM or PAR REC)
        json_file (string, optional): Absolute filepath to JSON sidecare
        
    Returns:
        eff_echo_sp (float): Effective Echo Spacing
        tot_read_time (float): Total Readout Time
        
    References:
    Approach 1: https://github.com/bids-standard/bids-specification/blob/master/src/04-modality-specific-files/01-magnetic-resonance-imaging-data.md
    Approach 2: https://osf.io/hks7x/ - page 7; 
    https://support.brainvoyager.com/brainvoyager/functional-analysis-preparation/29-pre-processing/78-epi-distortion-correction-echo-spacing-and-bandwidth
    
    Forum that raised this specific issue with Philips: https://neurostars.org/t/consolidating-epi-echo-spacing-and-readout-time-for-philips-scanner/4406
    
    Approaches 3 and 4 were found thorugh trial and error and yielded similar, but not the same values as approaches 1 and 2.
    '''
    
    # check file extension
    if 'dcm' in file:
        calc_method = 'dcm'
    elif 'PAR' in file:
        calc_method = 'par'
        
    # Create empty string variables
    bwpppe = ''
    recon_mat = ''
    pix_band = ''
    wfs = ''
    etl = ''
    red_fact = ''
        
    if calc_method.lower() == 'dcm':
        bwpppe = get_bwpppe(file)
        if json_file:
            recon_mat = get_recon_mat(json_file)
            pix_band = get_pix_band(json_file)
            # set bandwidth per pixel to empty if unknown
            try:
                if bwpppe.lower() == 'unknown':
                    bwpppe = ''
            except AttributeError:
                pass
            # set pixel bandwidth to empty if unknown
            try:
                if pix_band.lower() == 'unknown':
                    pix_band = ''
            except AttributeError:
                pass
            etl = recon_mat
    elif calc_method.lower() == 'par':
        wfs = get_wfs(file)
        etl = get_etl(file)
        red_fact = get_red_fact(file)
        # set water fat shift to empty if unknown
        try:
            if wfs.lower() == 'unknown':
                wfs = ''
        except AttributeError:
            pass
        # set echo train length to empty if unknown
        try:
            if etl.lower() == 'unknown':
                etl = ''
        except AttributeError:
            pass
         # set parallel reduction factor to empty if unknown
        try:
            if red_fact.lower() == 'unknown':
                red_fact = ''
        except AttributeError:
            pass
    
    # Calculate effective echo spacing and total readout time
    if bwpppe and recon_mat:
        eff_echo_sp = 1/(bwpppe * recon_mat)
        tot_read_time = eff_echo_sp * (recon_mat - 1)
    elif wfs and etl:
        if not red_fact:
            red_fact = 1
        eff_echo_sp = (((1000 * wfs)/(434.215 * (etl + 1)))/red_fact)
        tot_read_time = 0.001 * eff_echo_sp * etl
    elif pix_band and etl:
        eff_echo_sp = ((1/(pix_band * etl)) * (etl - 1)) * 1.3
        tot_read_time = eff_echo_sp * (etl - 1)
    elif pix_band and recon_mat:
        eff_echo_sp = ((1/(pix_band * recon_mat)) * (recon_mat - 1)) * 1.3
        tot_read_time = eff_echo_sp * (recon_mat - 1)
    else:
        eff_echo_sp = "unknown"
        tot_read_time = "unknown"
        
    return eff_echo_sp,tot_read_time

# `dcm2niix` Wrapper Function

In [64]:
def convert_image_data(file,basename,out_dir,cprss_lvl=6,bids=True,
                       anon_bids=True,gzip=True,comment=True,
                       adjacent=False,dir_search=5,nrrd=False,
                       ignore_2D=True,merge_2D=True,text=False,
                       progress=False,verbose=False,
                       write_conflicts="suffix",crop_3D=False,
                       lossless=False,big_endian="optimal",xml=False):
    '''
    Converts raw image data (DICOM, PAR REC, or Bruker) to NifTi (or NRRD) using dcm2niix.
    This is a wrapper function for dcm2niix (v1.0.20190902+). This wrapper functions has no returns, 
    however output files are generated in a specified directory that must exist prior to the 
    invokation of this function.
    
    Note: Most of the defaults for dcm2niix have been preserved aside from those starred (*) in the
    (optional) arguments section, in order to be BIDS compliant.

    Arguments (Required):
        file (string): Absolute path to raw image data file
        basename (string): Output file(s) basename
        out_dir (string): Absolute path to output directory (must exist at runtime)

    Arguments (Optional):
        cprss_lvl (int): Compression level [1 - 9] - 1 is fastest, 9 is smallest (default: 6)
        bids (bool): BIDS (JSON) sidecar (default: True) * 
        anon_bids (bool): Anonymize BIDS (default: True) * 
        gzip (bool): Gzip compress images (default: True) *
        comment (bool): Image comment(s) stored in NifTi header (default: True) *
        adjacent (bool): Assumes adjacent DICOMs/Image data (images from same series always in same folder) for faster conversion (default: False)
        dir_search (int): Directory search depth (default: 5)
        nrrd (bool): Export as NRRD instead of NifTi, not recommended (default: False)
        ignore_2D (bool): Ignore derived, localizer and 2D images (default: True)
        merge_2D (bool): Merge 2D slices from same series regardless of echo, exposure, etc. (default: True)
        text (bool): Text notes includes private patient details in separate text file (default: False)
        progress (bool): Report progress, slicer format progress information (default: True)
        verbose (bool): Enable verbosity (default: False)
        write_conflicts (string): Write behavior for name conflicts:
            - 'suffix' = Add suffix to name conflict (default)
            - 'overwrite' = Overwrite name conflict
            - 'skip' = Skip name conflict
        crop_3D (bool): crop 3D acquisitions (default: False)
        lossless (bool): Losslessly scale 16-bit integers to use dynamic range (default: True)
        big_endian (string): Byte order:
            - 'optimal' or 'native' = optimal/native byte order (default)
            - 'little-end' = little endian
            - 'big-end' = big endian
        xml (bool): Slicer format features (default: False)
        
        Returns:
            None
    '''

    # Empty list
    conv_cmd = list()

    # Get OS platform
    if platform.system().lower() == 'windows':
        conv_cmd.append("dcm2niix.exe")
    else:
        conv_cmd.append("dcm2niix")

    # Boolean True/False options arrays
    bool_opts = [bids, anon_bids, gzip, comment, adjacent, nrrd, ignore_2D, merge_2D, text, verbose, lossless, progress, xml]
    bool_vars = ["-b", "-ba", "-z", "-c", "-a", "-e", "-i", "-m", "-t", "-v", "-l", "--progress", "--xml"]

    # Initial option(s)
    if cprss_lvl:
        conv_cmd.append(f"-{cprss_lvl}")

    # Required option(s)
    if basename:
        conv_cmd.append("-f")
        conv_cmd.append(f"{basename}")

    if basename:
        conv_cmd.append("-f")
        conv_cmd.append(f"{basename}")

    if out_dir:
        conv_cmd.append("-o")
        conv_cmd.append(f"{out_dir}")

    # Keyword option(s)
    if write_conflicts.lower() == "suffix":
        conv_cmd.append("-w")
        conv_cmd.append("2")
    elif write_conflicts.lower() == "overwrite":
        conv_cmd.append("-w")
        conv_cmd.append("1")
    elif write_conflicts.lower() == "skip":
        conv_cmd.append("-w")
        conv_cmd.append("0")

    if big_endian.lower() == "optimal" or big_endian.lower() == "native":
        conv_cmd.append("--big_endian")
        conv_cmd.append("o")
    elif big_endian.lower() == "little-end":
        conv_cmd.append("--big_endian")
        conv_cmd.append("n")
    elif big_endian.lower() == "big-end":
        conv_cmd.append("--big_endian")
        conv_cmd.append("y")


    for idx,var in enumerate(bool_opts):
        if var:
            conv_cmd.append(bool_vars[idx])
            conv_cmd.append("y")

    # Required arguments
    # Filename
    conv_cmd.append("-f")
    conv_cmd.append(f"{basename}")

    # Output directory
    conv_cmd.append("-o")
    conv_cmd.append(f"{out_dir}")

    # Image file   
    conv_cmd.append(f"{file}")

    # System Call to dcm2niix (assumes dcm2niix is added to system path variable)
    subprocess.call(conv_cmd)
    
    return None